In [4]:
import urllib
import os
import datetime
import zipfile
import csv

def download_torontodata(http_url,download_file):
    """
    Download toronto csv
    :param download_folder:
    :return:
    """
#     if not os.path.exists(download_file):
#         os.makedirs(download_file)
    urllib.urlretrieve (http_url, download_file)

    print("Tornto CSV downloaded.")

TORONTO_URL = "http://opendata.toronto.ca/mls/business.licences/business.licences.csv"
TORNONTO_MERGED_FILE = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/merged_Toronto.csv"

download_torontodata(TORONTO_URL, TORNONTO_MERGED_FILE)

Tornto CSV downloaded.


In [26]:
import urllib
import os
import datetime
import zipfile
import csv

import sqlite3
import csv
SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Creating a new SQLite table with 1 column
c.execute("""CREATE TABLE IF NOT EXISTS AddressLatLongTor (
  IsUpdated integer,
  HouseStreet text,
  CityProvince text,
  PostalCode text,
  Latitude real,
  Longitude real,
  PRIMARY KEY (HouseStreet, CityProvince, PostalCode));
""")
c.execute("""CREATE INDEX IF NOT EXISTS queryindextor on AddressLatLongTor (HouseStreet, CityProvince,PostalCode);""")

# Committing changes and closing the connection to the database file
conn.commit()

def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def format_insert(housestreet, cityprovince, postalcode):
    
    if len(housestreet) > 0:
        sql_insert_template = "INSERT OR REPLACE INTO AddressLatLongTor"\
        "(IsUpdated, HouseStreet, CityProvince, PostalCode) " \
        "VALUES(0,'{0}','{1}','{2}');\n"

        return sql_insert_template.format(format_string(housestreet), 
                                          format_string(cityprovince), 
                                          format_string(postalcode))

    
def clean_str(val):
    val = val.replace(","," ").replace("\""," ")
    return val

def clean_vancouver_data(source_file, dest_file):
    
    f = csv.writer(open(dest_file, "w"))
    with open(source_file, "r") as filepointer:
        csvpointer = csv.reader(filepointer)
        csvpointer.next()
        linehead = "BusinessName,Status,IssuedDate,ExpiryDate,BusinessType,HouseStreet,CityProvince,PostalCode,Latitude,Longitude"
        f.writerow(linehead.split(","))
        for row in csvpointer:
            businessname = clean_str(row[4]) +" " +clean_str(row[1])
#             if businessname == "":
#                 businessname = clean_str(row[3])
            issuedate = clean_str(row[3])
            expirydate = clean_str(row[15])
            business_type = clean_str(row[0])
            housestreet = clean_str(row[7])
            cityprovince = clean_str(row[8])
            postalcode = clean_str(row[9])
            status = "Closed"
            if expirydate == "":
                status = "Open"
                expirydate = "31-DEC-2017"
            lon = ""
            lat = ""
            line = businessname+","+status+","+issuedate+","+ expirydate+","+business_type+","+housestreet+","+cityprovince+","+postalcode+","+lon+","+lat
            
            f.writerow([businessname, status, issuedate, expirydate, business_type, housestreet, cityprovince, postalcode, lat, lon])
#             sql_text = format_insert(housestreet, cityprovince, postalcode)
#             if sql_text != None:
#                 c.execute(sql_text)
    
    conn.commit()

    
TORNONTO_MERGED_FILE = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/merged_Toronto.csv"
TORONTO_CLEAN_FILE = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/clean_woll_Toronto.csv"

clean_vancouver_data(TORNONTO_MERGED_FILE, TORONTO_CLEAN_FILE)
conn.close()


In [17]:
import sqlite3
import csv
import urllib
import json

SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
#libraries for OAuth
import requests
from requests_oauthlib import OAuth1

def latlong_from_address(csvline):
     
    address = ' '.join(csvline)
    #keys
    client_key = 'dj0yJmk9ZUtvYVZYaURqNVNGJmQ9WVdrOVQzcEpVRlZ3TXpnbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD03NQ'
    client_secret = '59bfb1cbf4272ad5656a10b5660697781ba202b9'

    #Using OAuth1 to make a simple query request on when your application doesn't require any permissions
    baseurl = "https://query.yahooapis.com/v1/public/yql?"
    yql_query = "select centroid from geo.places where text=\""+address+"\""
    yql_url = baseurl + urllib.urlencode({'q':yql_query}) + "&format=json"
    queryoauth = OAuth1(client_key, client_secret, signature_type='query')
    r = requests.get(url=yql_url, auth=queryoauth)

    result = json.loads(r.content.decode('utf8'))
    latitude =0
    longitude = 0

    if result['query']['count'] > 1:
        latitude = result['query']['results']['place'][0]['centroid']['latitude']
        longitude = result['query']['results']['place'][0]['centroid']['longitude']
    elif result['query']['count'] == 1:
        latitude = result['query']['results']['place']['centroid']['latitude']
        longitude = result['query']['results']['place']['centroid']['longitude']
    elif result['query']['count'] == 0:
        print "nO resut"
    
    return latitude, longitude

def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def format_update(csvline, latitude, longitude):
    
    if len(csvline) > 0:
        sql_insert_template = """UPDATE AddressLatLongTor
        SET Latitude={0}, Longitude={1}, IsUpdated=1 
        WHERE HouseStreet='{2}'
        AND CityProvince='{3}'
        AND PostalCode ='{4}'
        """

        if len(csvline) > 0:
            housestreet = format_string(csvline[0])
            cityprovince = format_string(csvline[1])
            postalcode = format_string(csvline[2])

            return sql_insert_template.format(latitude, longitude,
                                              housestreet, cityprovince, postalcode)
        return ""



def update_record(csvline):
    lat, lon = latlong_from_address(csvline)
    sql_text = format_update(csvline, lat, lon)
    c.execute(sql_text)


total_count = 50000

    
def update_until_finish():   
    counter = 0
    csvline =['1']
    while counter < total_count:
        counter += 1
        c.execute("SELECT HouseStreet, CityProvince, PostalCode FROM AddressLatLongTor WHERE IsUpdated=0 LIMIT 1")
        csvline = c.fetchall()
        if len(csvline) > 0:
            csvline = list(csvline[0])
        else:
            break
        update_record(csvline)
        conn.commit()
    else:
        return "Done"

print update_until_finish()
conn.commit()
conn.close()

None


In [27]:
import csv
import sqlite3
import csv
SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

TORNONTO_FILE_WOLL = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/clean_woll_Toronto.csv"
TORNOTO_FINAL = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/clean_Toronto.csv"
def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def find_latlong(housestreet, cityprovince, postalcode):
    latitude=0
    longitude=0
    
    sql_text = """SELECT Latitude, Longitude 
    FROM AddressLatLongTor 
    WHERE HouseStreet='{0}' 
    AND CityProvince ='{1}'
    AND PostalCode = '{2}' LIMIT 1""".format(format_string(housestreet),
                 format_string(cityprovince), format_string(postalcode))
    #print sql_text
    c.execute(sql_text)
    csvline = c.fetchall()
    if len(csvline) > 0:
        latitude, longitude = csvline[0]
    #print latitude, longitude
    return latitude, longitude
        

def replace_latitude_longitude(filelocation, outputfile):
    f=csv.writer(open(outputfile, "w" ))
    
    with open(filelocation, 'r') as filepointer:
        csvreader = csv.reader(filepointer)
        headerline = csvreader.next()
        print(headerline)
        f.writerow(headerline)
        for csvline in csvreader:
            housestreet = csvline[5]
            cityprovince = csvline[6]
            postalcode = csvline[7]
            if len(csvline) < 9:
                print csvline
            latitude = csvline[8]
            longitude = csvline[9]
            if latitude == "" or longitude == "":
                latitude, longitude = find_latlong(housestreet, cityprovince, postalcode)
                if latitude != 0 or longitude != 0:
                    csvline[8] = str(latitude)
                    csvline[9] = str(longitude)
            f.writerow(csvline)
            #print csvline
            #break
                
replace_latitude_longitude(TORNONTO_FILE_WOLL, TORNOTO_FINAL)

['BusinessName', 'Status', 'IssuedDate', 'ExpiryDate', 'BusinessType', 'HouseStreet', 'CityProvince', 'PostalCode', 'Latitude', 'Longitude']
